# CSX scraping

In [190]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def launch_browser_and_access_iframe():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("useAutomationExtension", False)
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

    browser = webdriver.Chrome(options=chrome_options, executable_path='/Users/yann/Downloads/chromedriver')
    browser.get('http://www.intermodal.com/index.cfm/intermodal-maps/')
    # browser.set_window_size(960, 850)
    browser.maximize_window()
    time.sleep(3)
    # Set up a 15 seconds webdriver wait
    explicit_wait5 = WebDriverWait(browser, 5)

    #Close the first page
    button = browser.find_element_by_css_selector('#close-selector')
    button.click()

    # Center the map
    browser.execute_script('window.scrollBy(0, 194)') #194
    time.sleep(4)

    #Access the Iframe
    frame = browser.find_element_by_css_selector('#iframeMapSwitch')
    browser.switch_to.frame(0)
    
    return browser 

def hover_and_record_points(browser):
    explicit_wait5 = WebDriverWait(browser, 5)
    circles = explicit_wait5.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#terminalLyr_layer > image')))

    data = []
    time.sleep(3)
    print(len(circles))
    for circle in circles:
        # Avoid going out of bounds
        if circle.location["y"] > 680:
            # One US terminal should still be accounted
            if circle.location["y"] == 694:
                data.append('Rio Valley, TX')
            continue
            
        # Execute mouseover on the element, go under and over again
        ActionChains(browser).move_to_element(circle).perform()
        ActionChains(browser).move_by_offset(0,15).perform()
        ActionChains(browser).move_to_element(circle).perform()

        time.sleep(2)
        # Print the data
        point_text = explicit_wait5.until(EC.visibility_of_element_located((By.CSS_SELECTOR,
                                                                            '#map_root > div.esriPopup > div.esriPopupWrapper > div.sizer.content > div'))).text
        print(point_text)

        data.append(point_text)
        
    return data

def loop_each_state(browser, record_points=False):
    explicit_wait5 = WebDriverWait(browser, 5)
    df = pd.DataFrame()
    time.sleep(2)
    states = explicit_wait5.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#state > option')))
    states = states[2:]
    
    for state in states[:1]:
        browser.find_element_by_xpath(f"//select[@name='state']/option[text()='{state.text}']").click()
        time.sleep(3)

        # Wait for all circles to load
        cities = explicit_wait5.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#city > option')))
        cities = cities[2:]

        for city in cities[:2]:
            print(city.text)
            #errorMsg
            try:
                browser.find_element_by_css_selector('#errorMsg')
                continue
            except:
                browser.find_element_by_xpath(f"//select[@name='city']/option[text()='{city.text}']").click()
                time.sleep(5)

                browser.find_element_by_css_selector('#t17_divButton3').click()
                time.sleep(5)
                if record_points:
                    data = hover_and_record_points(browser)
                    df = df.append({'state':state.text,'city':city.text,'data':data},ignore_index=True)
                    time.sleep(2)
    return df

def scrap_csx():
    browser = launch_browser_and_access_iframe()
    df = loop_each_state(browser, record_points=True)
    return df

In [189]:
df = scrap_csx()

Bessemer
17
292
Portland, OR
408
Sparks, NV
409
Sparks, NV
433
Lathrop, CA
257
Seattle, WA
262
Seattle, WA

Tacoma, WA
430
Oakland, CA
439
Denver, CO
482
Las Vegas, NV
645
Central Florida ILC
427
Baltimore, MD
556
Bessemer (CAICTF), AL
552
Bessemer (CAICTF), AL
403
Chicago (59th Street), IL
601
Jacksonville, FL
679
Miami, FL
393
Kearny, NJ


In [192]:
df.loc[0,'data']

['Portland, OR',
 'Sparks, NV',
 'Sparks, NV',
 'Lathrop, CA',
 'Seattle, WA',
 'Seattle, WA\n\nTacoma, WA',
 'Oakland, CA',
 'Denver, CO',
 'Las Vegas, NV',
 'Central Florida ILC',
 'Baltimore, MD',
 'Bessemer (CAICTF), AL',
 'Bessemer (CAICTF), AL',
 'Chicago (59th Street), IL',
 'Jacksonville, FL',
 'Miami, FL',
 'Kearny, NJ']

In [193]:
df

,city,data,state
0,Bessemer,"[Portland, OR, Sparks, NV, Sparks, NV, Lathrop...",Alabama


# Experiments

In [229]:
def test_state_city_combo(browser, states, record_points=True):
    explicit_wait5 = WebDriverWait(browser, 5)
    df = pd.DataFrame()
    time.sleep(2)

    
    for state in states:
        browser.find_element_by_xpath(f"//select[@name='state']/option[text()='{state}']").click()
        time.sleep(3)

        # Wait for all circles to load
        cities = explicit_wait5.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#city > option')))
        print(cities)
        cities = cities[2:]

        for city in cities[:2]:
            
            browser.find_element_by_xpath(f"//select[@name='city']/option[text()='{city.text}']").click()
            time.sleep(5)

            browser.find_element_by_css_selector('#t17_divButton3').click()
            time.sleep(5)
            
            print(city.text)
            error = browser.find_element_by_css_selector('#errorMsg').text
            print('EEEEEEEEERR   :',error)
            if error != '':
                continue
                
            else:

                if record_points:
                    data = hover_and_record_points(browser)
                    df = df.append({'state':state,'city':city.text,'data':data},ignore_index=True)
                    time.sleep(2)
    return df, error

In [230]:
browser = launch_browser_and_access_iframe()
df, error = test_state_city_combo(browser=browser, states=['Missouri','Alabama'])#'Alabama',

[<selenium.webdriver.remote.webelement.WebElement (session="903eeae1f7b3b1ded8b38979097fca06", element="5abdcc60-ca15-43fb-841b-aa8e77468846")>, <selenium.webdriver.remote.webelement.WebElement (session="903eeae1f7b3b1ded8b38979097fca06", element="7651fe4a-d4ba-4a41-8450-445986dd9699")>, <selenium.webdriver.remote.webelement.WebElement (session="903eeae1f7b3b1ded8b38979097fca06", element="d4d86b9e-58e7-490f-bdfe-6a3b37db295f")>]
Saint Louis
EEEEEEEEERR   : This is not an origin terminal. Please select another.
[<selenium.webdriver.remote.webelement.WebElement (session="903eeae1f7b3b1ded8b38979097fca06", element="0e64527b-e55f-4681-aabe-ac931c9bbfec")>, <selenium.webdriver.remote.webelement.WebElement (session="903eeae1f7b3b1ded8b38979097fca06", element="89b3ecf1-2927-423d-b7fb-bec1f6a29f20")>, <selenium.webdriver.remote.webelement.WebElement (session="903eeae1f7b3b1ded8b38979097fca06", element="83a2f98d-fea1-415f-994c-d2c6823a7c52")>]
Bessemer
EEEEEEEEERR   : 
17
292
Portland, OR
408
S

KeyboardInterrupt: 

In [221]:
df

,city,data,state
0,Bessemer,"[Portland, OR, Sparks, NV, Sparks, NV, Lathrop...",Alabama
1,Saint Louis,"[Oakland, CA, Oakland, CA, Oakland, CA, Los An...",Missouri


In [214]:
a = error.text

In [216]:
a == ''

True

## Load the initial browser

In [32]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
browser = webdriver.Chrome(options=chrome_options, executable_path='/Users/yann/Downloads/chromedriver')
browser.get('http://www.intermodal.com/index.cfm/intermodal-maps/')
browser.maximize_window()

# Set up a 30 seconds webdriver wait
explicit_wait30 = WebDriverWait(browser, 10)

# finding the button using ID
button = browser.find_element_by_css_selector('#close-selector')

# clicking on the button
button.click()
# browser.execute_script('window.scrollBy(0, 250)')
frame = browser.find_element_by_css_selector('#iframeMapSwitch')
browser.switch_to.frame(0)
browser.implicitly_wait(10)
b = browser.find_element_by_css_selector('#terminalLyr_layer > image')

circles = explicit_wait30.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#terminalLyr_layer > image')))

In [45]:
browser.switch_to.default_content()
browser.execute_script('window.scrollBy(0, 300)')

In [46]:
browser.switch_to.frame(0)

In [47]:
ActionChains(browser).move_to_element(circles[0]).perform()
ActionChains(browser).move_to_element(circles[5]).perform()

In [53]:
circles = explicit_wait30.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#terminalLyr_layer > image')))

## Find a way to hover over the points 

In [48]:
circles[0]

<selenium.webdriver.remote.webelement.WebElement (session="6fce98daa7a503342e9527b3c5e7aa9e", element="014764e6-7f1d-4b55-a85c-1c0505aa0b6c")>

In [49]:
circles[3].location.get("y")

512

In [50]:
circles[0].location.get('y')

430

In [51]:
# browser.execute_script("const mouseoverEvent = new Event('mouseover');arguments[0].dispatchEvent(mouseoverEvent)", circles[20])
ActionChains(browser).move_to_element(circles[0]).perform()
ActionChains(browser).move_by_offset(0,25).perform()
ActionChains(browser).move_by_offset(0,-15).perform()
# browser.execute_script("const mouseoverEvent = new Event('mouseover');arguments[0].dispatchEvent(mouseoverEvent)", circles[20])
ActionChains(browser).move_to_element(circles[0]).perform()
time.sleep(3)
browser.implicitly_wait(10)
print(explicit_wait30.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#map_root > div.esriPopup > div.esriPopupWrapper > div.sizer.content > div > a'))).text)

Portland, OR


In [106]:

import time

# browser.execute_script("const mouseoverEvent = new Event('mouseover');arguments[0].dispatchEvent(mouseoverEvent)", circles[20])
ActionChains(browser).move_to_element(circles[0]).perform()
# ActionChains(browser).move_by_offset(0,25).perform()
# ActionChains(browser).move_by_offset(0,-15).perform()
# # browser.execute_script("const mouseoverEvent = new Event('mouseover');arguments[0].dispatchEvent(mouseoverEvent)", circles[20])
# ActionChains(browser).move_to_element(circles[0]).perform()
time.sleep(3)
browser.implicitly_wait(10)
print(explicit_wait30.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#map_root > div.esriPopup > div.esriPopupWrapper > div.sizer.content > div > a'))).text)
# browser.execute_script("const mouseoverEvent = new Event('mouseover');arguments[0].dispatchEvent(mouseoverEvent)", circles[4])
ActionChains(browser).move_to_element(circles[5]).perform()
# ActionChains(browser).move_by_offset(0,25).perform()
# ActionChains(browser).move_by_offset(0,-15).perform()
# ActionChains(browser).move_to_element(circles[5]).perform()
browser.implicitly_wait(10)
time.sleep(3)
print(explicit_wait30.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#map_root > div.esriPopup > div.esriPopupWrapper > div.sizer.content > div > a'))).text)
print(explicit_wait30.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#map_root > div.esriPopup > div.esriPopupWrapper > div.sizer.content > div'))).text)
# browser.execute_script("const mouseoverEvent = new Event('mouseover');arguments[0].dispatchEvent(mouseoverEvent)", circles[7])
ActionChains(browser).move_to_element(circles[8]).perform()
# ActionChains(browser).move_by_offset(0,25).perform()
# ActionChains(browser).move_by_offset(0,-15).perform()
# ActionChains(browser).move_to_element(circles[8]).perform()
time.sleep(3)
print(explicit_wait30.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#map_root > div.esriPopup > div.esriPopupWrapper > div.sizer.content > div > a'))).text)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=56952): Max retries exceeded with url: /session/e2829d59ce45719af5b13a53acb27199/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd264be6dc0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [91]:
import pandas as pd

## Final code

In [152]:
len(circles)

59

In [154]:
len(data)

57

In [155]:
data

['Oakland, CA',
 'Oakland, CA',
 'Oakland, CA',
 'Los Angeles LATC\n\nLos Angeles, CA\n\nCity of Industry, CA',
 'Los Angeles LATC\n\nLos Angeles, CA\n\nCity of Industry, CA',
 'Los Angeles LATC\n\nLos Angeles, CA\n\nCity of Industry, CA',
 'Sparks, NV',
 'City of Industry, CA\n\nSan Bernardino, CA',
 'Portland, OR\n\nPortland, OR',
 'Portland, OR\n\nPortland, OR',
 'Tacoma, WA\n\nSeattle, WA\n\nSouth Seattle, WA',
 'Tacoma, WA\n\nSeattle, WA\n\nSouth Seattle, WA',
 'Tacoma, WA\n\nSeattle, WA\n\nSouth Seattle, WA',
 'Las Vegas, NV',
 'Salt Lake City, UT',
 'Santa Teresa Intermodal Ramp',
 'Denver, CO',
 'Laredo, TX',
 'San Antonio, TX',
 'Dallas, TX',
 'Houston, TX',
 'Chicago (Corwith), IL\n\nChicago (59th Street), IL\n\nChicago (Bedford Park), IL\n\nChicago (Cicero), IL',
 'Chicago (Corwith), IL\n\nChicago (59th Street), IL\n\nChicago (Bedford Park), IL\n\nChicago (Cicero), IL',
 'Bessemer (CAICTF), AL',
 'Nashville, TN',
 'Indianapolis, IN',
 'Louisville, KY',
 'Cincinnati, OH',
 'F

In [138]:
frame = browser.find_element_by_css_selector('#map_intermodal')
browser.switch_to.frame(0)
browser.execute_script('window.scrollBy(0, 264)') #194

In [119]:
data = []
df = pd.DataFrame({})
time.sleep(5)
for circle in circles:
    df = df.append({'x':circle.location["x"],'y':circle.location["y"]},ignore_index=True)

In [120]:
df

,x,y
0,99.0,430.0
1,114.0,433.0
2,116.0,432.0
3,140.0,512.0
4,140.0,513.0
5,144.0,514.0
6,143.0,408.0
7,154.0,514.0
8,133.0,291.0
9,134.0,292.0


In [103]:
data = []
frame = browser.find_element_by_css_selector('#iframeMapSwitch')
browser.switch_to.default_content()
ActionChains(browser).move_to_element(frame).perform()
for circle in df.itertuples():
    ActionChains(browser).move_by_offset(circle.x,circle.y).perform()
    ActionChains(browser).move_by_offset(0,15).perform()
    ActionChains(browser).move_by_offset(0,-15).perform()
    browser.switch_to.frame(0)
    listing = explicit_wait15.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#map_root > div.esriPopup > div.esriPopupWrapper > div.sizer.content > div'))).text
    print(listing)
    
    data.append(listing)
    
    browser.switch_to.default_content()
    ActionChains(browser).move_to_element(frame.perform())

MoveTargetOutOfBoundsException: Message: move target out of bounds
  (Session info: chrome=87.0.4280.88)


In [ ]:
ActionChains(browser).move_to_element_with_offset

In [86]:
print(circles[0].location["x"])
print(circles[0].location["y"])

99
430


In [89]:
browser.switch_to.default_content()
browser.execute_script('window.scrollBy(0, 194)')
frame = browser.find_element_by_css_selector('#iframeMapSwitch')
browser.switch_to.frame(0)

In [90]:
print(circles[0].location["x"])
print(circles[0].location["y"])

99
430


In [74]:
print(frame.location["x"])
print(frame.location["y"])

240
194


In [ ]:
#map_gc

## Loop over the states and cities

In [ ]:
#country -> USA
#state -> interate

States = explicit_wait30.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#state > option:')))
States = States[2:]

### Inspiration example 

In [95]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
browser = webdriver.Chrome(options=chrome_options, executable_path='/Users/yann/Downloads/chromedriver')
browser.get('http://insideairbnb.com/melbourne/')
browser.maximize_window()

# Set up a 30 seconds webdriver wait
explicit_wait30 = WebDriverWait(browser, 30)

try:
    # Wait for all circles to load
    circles = explicit_wait30.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'svg.leaflet-zoom-animated > g:nth-child(2) > circle')))
except TimeoutException:
    browser.refresh()

data = []
for circle in circles:
    # Execute mouseover on the element
    browser.execute_script("const mouseoverEvent = new Event('mouseover');arguments[0].dispatchEvent(mouseoverEvent)", circle)
    # Wait for the data to appear
    listing = explicit_wait30.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#listingHover')))
    # listing now contains the full element list - you can parse this yourself and add the necessary data to `data`
    # Close the listing
    browser.execute_script("arguments[0].click()", listing.find_element_by_tag_name('button'))